In [ ]:
import os
import shutil
import subprocess
import numpy as np

def run_geant4_simulation(mineral_name, particle_type, n_events, energies_gev):
    """
    Dynamically creates a Geant4 macro file and runs the simulation for a given list of energies.

    Args:
        mineral_name (str): Name of the target material (must be defined in Geant4).
        particle_type (str): Particle type for the gun (e.g., "mu-", "proton").
        n_events (int): Number of primary particles to simulate per energy.
        energies_gev (np.array): Array of energies in GeV to simulate.
    """
    
    # 1. Define Directories
    # The C++ code is hardcoded to write to ./output/
    # We will move the files to Data/Geant4_data/{mineral_name}/
    default_output_dir = "./build/output"
    final_output_dir = f"./{mineral_name}_{particle_type}"
    
    if not os.path.exists(default_output_dir):
        os.makedirs(default_output_dir)
    
    if not os.path.exists(final_output_dir):
        os.makedirs(final_output_dir)

    # 2. Construct the Macro Content
    macro_content = f"""# Auto-generated macro file
/control/verbose 0
/run/verbose 0
/tracking/verbose 0

# --- Geometry Setup ---
/testhadr/TargetMat        {mineral_name}
/testhadr/TargetRadius     10 m
/testhadr/TargetLength     10 m

# --- Physics ---
/run/setCut                0.005 mm
/run/initialize

# --- Run Loop ---
/gun/particle {particle_type}
"""

    # Append the run commands for each energy in the array
    for energy in energies_gev:
        macro_content += f"\n/gun/energy {energy:.16f} GeV\n"
        macro_content += f"/run/beamOn {n_events}\n"

    # 3. Write the Macro File
    macro_filename = "./build/run.in"
    with open(macro_filename, "w") as f:
        f.write(macro_content)

    # 4. Run Geant4
    # Assuming the executable is named 'Hadr01' and is in the current directory
    executable = "./build/Run"
    
    if not os.path.exists(executable):
        print(f"Error: Executable '{executable}' not found. Compile the C++ code first.")
        return

    print(f"Starting Geant4 simulation for {mineral_name} with {len(energies_gev)} energy points...")
    
    try:
        # Capture output to avoid cluttering the notebook/console
        result = subprocess.run([executable, macro_filename], check=True, capture_output=True, text=True)
        print(result.stdout) # Uncomment to see Geant4 output if needed
        
    except subprocess.CalledProcessError as e:
        print("Error running Geant4:")
        print(e.stderr)
        return

    # 5. Move and Rename Files
    # The C++ RunAction writes files named outNuclei_{RunID}.txt
    # RunID corresponds to the index in the energies_gev array (0, 1, 2...)
    
    print(f"Simulation finished. Moving files to {final_output_dir}...")
    
    for i, energy in enumerate(energies_gev):
        # The C++ RunID starts at 0 for the first /run/beamOn
        src_filename = f"outNuclei_{i}.txt"
        src_path = os.path.join(default_output_dir, src_filename)
        
        # We rename it to include the energy for clarity: outNuclei_{energy}.txt
        dst_filename = f"outNuclei_{energy:.6f}.txt"
        dst_path = os.path.join(final_output_dir, dst_filename)
        
        if os.path.exists(src_path):
            shutil.move(src_path, dst_path)
        else:
            print(f"Warning: Expected output file {src_filename} not found.")

    print("All done.")


In [17]:
energies = np.logspace(-2, 4, 3)
run_geant4_simulation("Halite", "mu+", 10, energies)

Starting Geant4 simulation for Halite with 3 energy points...

**************************************************************
 Geant4 version Name: geant4-11-03-patch-02 [MT]   (25-April-2025)
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************

Visualization Manager instantiating with verbosity "warnings (3)"...
Visualization Manager initialising...
Registering graphics systems...

You have successfully registered the following graphics systems.
Registered graphics systems are:
  ASCIITree (ATree)
  DAWNFILE (DAWNFILE)
  G4HepRepFile (HepRepFile)
  RayTracer (RayTracer)
  VRML2FILE (VRML2FILE)
  gMocrenFile (gMocrenFile)
  TOOLSSG_OFFSCREEN (TSG_OFFSCREEN, TSG_FILE)
  OpenGLImmediateQ